In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# from skle.preprocessing imp

from sklearn.linear_model import LogisticRegression, ridge_regression
from sklearn.ensemble import RandomForestRegressor #
from sklearn.svm import SVR
from xgboost import XGBRegressor ##
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


In [2]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
combined = [train_data,test_data]

In [3]:
train_data.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,target
0,924,3.000000,70.064256,24.020824,44.508426,24.310492,3.117595,cat2,398.455102,cat13,1416.142572
1,956,4.898979,56.267220,20.591260,32.878564,20.000000,2.886815,cat2,70.823087,cat2,1416.142572
2,1583,4.242641,94.620294,32.771939,54.872580,31.511903,3.276767,cat2,770.742788,cat11,1416.142572
3,1586,4.000000,87.641314,30.757113,52.697249,30.133038,3.086600,cat2,751.995992,cat13,1416.142572
4,3487,3.162278,87.549986,31.606961,59.304300,31.591138,3.298560,cat2,695.048624,cat5,1416.142572


In [4]:
train_data.describe()

,col0,col1,col2,col3,col4,col5,col6,col8,target
count,15000.000000,15000.000000,15000.000000,14856.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7500.500000,5.244599,48.152720,21.952897,35.810135,21.143822,1.895364,405.415168,412.128917
std,4330.271354,1.253538,17.481893,7.712690,12.474110,7.313136,0.452278,226.439124,372.613689
min,1.000000,1.000000,1.414214,1.414214,1.732051,1.414214,0.707036,10.019787,-199.453698
25%,3750.750000,4.358899,37.762415,17.146428,28.089144,16.643317,1.581139,208.007229,137.325117
50%,7500.500000,5.477226,45.628938,20.784610,34.117444,20.149442,1.855748,409.101140,319.536975
75%,11250.250000,6.082763,55.542776,25.396850,41.677332,24.474477,2.156386,597.959193,571.369087
max,15000.000000,7.211103,198.292713,80.280757,169.014792,77.987178,3.872996,799.963000,1416.142572


In [5]:
cat_features = train_data.select_dtypes(['object']).columns
cat_features

Index(['col7', 'col9'], dtype='object')

In [6]:
for col in cat_features:
    print('\n', train_data[col].nunique())


 5

 13


In [7]:
for dataset in combined:
    dataset['col7'] = dataset['col7'].fillna('cat2')

In [8]:
for dataset in combined:
    dataset['col9'].fillna('cat10',inplace = True)

In [9]:
train_data['col3'].fillna(train_data['col3'].mean(),inplace = True)
test_data['col3'].fillna(test_data['col3'].mean(),inplace = True)

In [10]:
train_data = pd.get_dummies(train_data, prefix = ['col7'], columns = ['col7'])
test_data = pd.get_dummies(test_data, prefix = ['col7'], columns = ['col7'])

In [11]:
train_data = pd.get_dummies(train_data, prefix = ['col9'], columns = ['col9'])
test_data = pd.get_dummies(test_data, prefix = ['col9'], columns = ['col9'])

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   col0        15000 non-null  int64  
 1   col1        15000 non-null  float64
 2   col2        15000 non-null  float64
 3   col3        15000 non-null  float64
 4   col4        15000 non-null  float64
 5   col5        15000 non-null  float64
 6   col6        15000 non-null  float64
 7   col8        15000 non-null  float64
 8   target      15000 non-null  float64
 9   col7_cat1   15000 non-null  uint8  
 10  col7_cat2   15000 non-null  uint8  
 11  col7_cat3   15000 non-null  uint8  
 12  col7_cat4   15000 non-null  uint8  
 13  col7_cat6   15000 non-null  uint8  
 14  col9_cat10  15000 non-null  uint8  
 15  col9_cat11  15000 non-null  uint8  
 16  col9_cat12  15000 non-null  uint8  
 17  col9_cat13  15000 non-null  uint8  
 18  col9_cat14  15000 non-null  uint8  
 19  col9_cat15  15000 non-nul

In [13]:
train_data.describe()

,col0,col1,col2,col3,col4,col5,col6,col8,target,col7_cat1,...,col9_cat13,col9_cat14,col9_cat15,col9_cat2,col9_cat3,col9_cat4,col9_cat5,col9_cat6,col9_cat8,col9_na
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,...,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7500.500000,5.244599,48.152720,21.952897,35.810135,21.143822,1.895364,405.415168,412.128917,0.099133,...,0.066267,0.070133,0.071933,0.071867,0.071200,0.069467,0.071333,0.074400,0.074800,0.074800
std,4330.271354,1.253538,17.481893,7.675577,12.474110,7.313136,0.452278,226.439124,372.613689,0.298851,...,0.248756,0.255380,0.258386,0.258276,0.257167,0.254255,0.257389,0.262429,0.263077,0.263077
min,1.000000,1.000000,1.414214,1.414214,1.732051,1.414214,0.707036,10.019787,-199.453698,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3750.750000,4.358899,37.762415,17.175564,28.089144,16.643317,1.581139,208.007229,137.325117,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7500.500000,5.477226,45.628938,20.856654,34.117444,20.149442,1.855748,409.101140,319.536975,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11250.250000,6.082763,55.542776,25.337719,41.677332,24.474477,2.156386,597.959193,571.369087,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,15000.000000,7.211103,198.292713,80.280757,169.014792,77.987178,3.872996,799.963000,1416.142572,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
cols = train_data.loc[:,train_data.std() > 2].columns.values
cols = cols[:len(cols) - 1]
cols

array(['col0', 'col2', 'col3', 'col4', 'col5', 'col8'], dtype=object)

In [15]:
sc = StandardScaler()
for c in cols:
    train_data[c] = sc.fit_transform(train_data[c].values.reshape(-1,1))
    test_data[c] = sc.fit_transform(test_data[c].values.reshape(-1,1))

In [16]:
y = train_data.loc[:,'target'].astype(int)
X = train_data.drop('target',axis = 1)
X.columns

Index(['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col8',
       'col7_cat1', 'col7_cat2', 'col7_cat3', 'col7_cat4', 'col7_cat6',
       'col9_cat10', 'col9_cat11', 'col9_cat12', 'col9_cat13', 'col9_cat14',
       'col9_cat15', 'col9_cat2', 'col9_cat3', 'col9_cat4', 'col9_cat5',
       'col9_cat6', 'col9_cat8', 'col9_na'],
      dtype='object')

In [17]:
test_data['col7_cat4'] = 0

In [18]:
x_train,x_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state = 42)


In [21]:
model = LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=1800,
                                learning_rate=0.17, n_estimators=100, max_depth=2500,
                                metric='rmse', bagging_fraction=0.7, feature_fraction=0.8, reg_lambda=0.9)
# model = CatBoostRegressor(iterations = 100,
#                           learning_rate = 0.1,
#                           depth = 15)
# model = LGBMRegressor()
clf = model.fit(x_train,y_train)


In [22]:
np.sqrt(mean_squared_error(y_train,clf.predict(x_train)))

28.92948971555465

In [23]:
np.sqrt(mean_squared_error(y_val,clf.predict(x_val)))

160.05991672597762

In [24]:
cross_val_score(model,x_val,y_val,scoring='neg_mean_squared_error',cv = 7).mean()

-32328.569025279452

In [25]:
y_pred = clf.predict(test_data)
y_pred

array([ 741.54958063, 1224.84608968,  818.10613588, ...,  397.14362301,
        460.28021511,  343.90950949])

In [38]:
sub = pd.DataFrame(y_pred,index = None,columns=['target'])
sub

,target
0,797.924378
1,1309.517570
2,850.245309
3,486.490919
4,474.041824
...,...
5635,464.435579
5636,440.818180
5637,497.695903
5638,520.585686


In [40]:
sub.to_csv('himanshu_madan_submission.csv')